# LABRST-2678 Programming for Network Engineers

## Lab Tasks

### Task 1: Building blocks of Netmiko python script

In this lab exercise you will learn how a basic netmiko python script is constructed. Think of it like a Lego block, first we will start with basic building blocks and quickly iterate through it to build complex programming logics. 

Let’s dive into the basic building blocks.

First we have to import the Netmiko connection libraries
```python
from netmiko import Netmiko
```
The connection parameters are collected in a Python dictionary. The connection parameters provide Netmiko with everything it needs to create the SSH connection. In the below example we have shown how to connect to a IOS device, if it is a NXOS device, simply change the device type to 'device_type': 'cisco_nxos'.

```python
ios1 = {
    'device_type': 'cisco_ios',
    'ip': '192.18.1.55',
    'username': 'cisco',
    'password': 'cisco',
}
```
Netmiko is a function that calls the necessary connection parameters and device type (cisco_ios, cisco_xr, cisco_nxos, etc.) Once connection parameters are loaded, the script will launch a SSH connection to login into the device.

```python
net_connect = Netmiko(**ios1)
```
.send_command() method is used to send show commands over the channel and receive the output back. Here, we are reading the output of ‘show version’ command and storing it in a variable named 'output'.

```python
output = net_connect.send_command('show version’)
```
Using the .send_config_set() method, we can program the network device to enter into configuration mode and make configuration changes. After executing the config_commands the script will exit the configuration mode.

```python
output = net_connect.send_config_set(config_commands)
```

We can send either only one command or multiple lines of commands by converting it into a simple list. If we are sending a big configuration, it is recommended to use the .send_config_from_file() method. 

```python
output = net_connect.send_config_from_file('more_config')
```

All of the session output is stored in an output variable and then printed out in the screen for our reference. 

For more information on the all the connection methods avaialvle with Netmiko, pls refer the documentation

__[Netmiko Introduction](https://pynet.twb-tech.com/blog/automation/netmiko.html)__

__[Netmiko Documentation](https://netmiko.readthedocs.io/en/latest/classes/base_connection.html)__


### Task 2: Executing a show command on a network device

The python script will
- login to the iosv-1 device via SSH
- run the show version command
- capture the output
- print the command output to the screen.

In [1]:
#!/usr/bin/env python
from netmiko import Netmiko

# SSH Connection Details
ios1 = {
    'device_type': 'cisco_ios',
    'ip': '172.21.56.120',
    'username': 'cisco',
    'password': 'cisco',
}

# Establish SSH to device and run show command
net_connect = Netmiko(**ios1)
output = net_connect.send_command('show version')
print (output)

NetMikoTimeoutException: Connection to device timed-out: cisco_ios 172.21.56.120:22

### Task 3: Configuring a network device

The python script will
- login to the iosv-1 device via SSH
- make a configuration change "logging host 10.1.1.1"
- capture the output
- print the command output to the screen.

Notice we are reusing the same code, that we used in previous task. Only change is, instead of net_connect.send_command() we are using net_connect.send_config_set(). Rest of the script remains the same. Moving forward we will use the same logic, iterate the throught the code we have already written and add addtional logic on top of it to automate complex network tasks. 

In [3]:
#!/usr/bin/env python
from netmiko import Netmiko

# SSH Connection Details
ios1 = {
    'device_type': 'cisco_ios',
    'ip': '172.21.56.120',
    'username': 'cisco',
    'password': 'cisco',
}

# Establish SSH to device and run config command
net_connect = Netmiko(**ios1)
output = net_connect.send_config_set('logging host 10.1.1.1')
print (output)

config term
Enter configuration commands, one per line.  End with CNTL/Z.
asr1(config)#logging host 1.1.1.1
asr1(config)#end
asr1#


### Task 4: Configuring mutiple network devices

The python script will
- login to the iosv-1 and iosv-2 devices via SSH
- make a configuration change "logging host 10.1.1.2"
- capture the output
- print the command output to the screen.

To configure multiple devices, we have to create multiple SSH connection profiles and add it to a list. Then add a for loop to iterate through the connection profiles and make config changes to the IOS devices.

```python
device_list = [ios1, ios2]

for device in device_list:
    ** Netmiko config code block **
```

In [5]:
#!/usr/bin/env python
from netmiko import Netmiko

# SSH Connection Details
ios1 = {
    'device_type': 'cisco_ios',
    'ip': '172.21.56.120',
    'username': 'cisco',
    'password': 'cisco',
}

ios2 = {
    'device_type': 'cisco_ios',
    'ip': '172.21.56.121',
    'username': 'cisco',
    'password': 'cisco',
}

devices = [ios1, ios2]

for device in devices: 
    # Establish SSH to device and run config command
    net_connect = Netmiko(**device)
    output = net_connect.send_config_set('logging host 10.1.1.2')
    print (output)

config term
Enter configuration commands, one per line.  End with CNTL/Z.
asr1(config)#logging host 1.1.1.1
asr1(config)#end
asr1#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
asr2(config)#logging host 1.1.1.1
asr2(config)#end
asr2#


### Task 5:  Pushing large configurations across multiple devices

Now we have a solid understanding of how to write basic scripts. Next step is to make our code modular. For that we should remove all of the hardcoded variables from the script. The variable that are used in the script needs to be provided by either the user who runs the script or from a file.

The python script will
- Load the device ip details from a text file named 'device_list'
- Load the configuration commands from a text file named 'config_commands'
- Requests the login credentials from the user
- Uses getpass() module to encrypt the user provided password
- Login to each device in the 'device_list' and configure the commands given in 'config_commands' file.
- Print the output


First we use the input() and getpass() modules to collect the login credentials. Next, read the contents of the file using inbuilt python file module - with open(). After that we loop through the device_list and configure the devives. 

```python
username = input()
password = getpass()

with open('file_name') as f:
    device_list = f.read().splitlines()

for devices in device_list:
    ** Netmiko config code block **    
```

In [7]:
#!/usr/bin/env python
from netmiko import Netmiko
from getpass import getpass

# SSH username and password provided by user
username = input('Enter your SSH username: ')
password = getpass('Enter your password: ')

# Sending device ip's stored in a file 
with open('device_list') as f:
    device_list = f.read().splitlines()

# Iterate through device list and configure the devices 
for device in device_list:
    print ('Connecting to device ' + device)
    ip_address_of_device = device
    
    # SSH Connection details
    ios_device = {
        'device_type': 'cisco_ios',
        'ip': ip_address_of_device, 
        'username': username,
        'password': password
    }
 
    net_connect = Netmiko(**ios_device)
    output = net_connect.send_config_from_file('config_commands')
    print (output)        

Enter your SSH username: cisco
Enter your password: ········
Connecting to device 172.21.56.120
config term
Enter configuration commands, one per line.  End with CNTL/Z.
asr1(config)#logging console
asr1(config)#logging host 1.1.1.2
asr1(config)#ntp server 1.1.1.3
asr1(config)#ip name-server 1.1.1.4
asr1(config)#no ip http server
asr1(config)#no ip http secure-server
asr1(config)#end
asr1#
Connecting to device 172.21.56.121
config term
Enter configuration commands, one per line.  End with CNTL/Z.
asr2(config)#logging console
asr2(config)#logging host 1.1.1.2
asr2(config)#ntp server 1.1.1.3
asr2(config)#ip name-server 1.1.1.4
asr2(config)#no ip http server
asr2(config)#no ip http secure-server
asr2(config)#end
asr2#


### Task 5:  Error handling and verification

In this task we see show to enable error handling for our scripts. The idea behind error handling is to catch any exceptions that occurs during the execution of the script. Without error handling when an exception is detected the python script terminates and reports error.  



Try and expect code blocks will help us to catch any errors during program execution. We have added diffrent exceptions that can be triggered during the exceution. For example: device timeout, reachability issues, wrong user credential errors, etc. If an exception is detected, the script will move on to next device and complete the task. 

```python
username = input()
password = getpass()

with open('file_name') as f:
    device_list = f.read().splitlines()

for devices in device_list:
    try:
        ** Netmiko connection **
    except:
        ** Error condition **
        continue
    
    ** Netmiko config code block **    
```

Test the exceptions, try providing wrong ip in device_list or wrong username and password to check whether the scripts catch and report the exceptions.

In [8]:
#!/usr/bin/env python
from getpass import getpass
from netmiko import Netmiko
from netmiko.ssh_exception import NetMikoTimeoutException
from paramiko.ssh_exception import SSHException
from netmiko.ssh_exception import AuthenticationException

# Collect login credentials
username = input('Enter your SSH username: ')
password = getpass('Enter your password: ')

# Sending device ip's stored in a file
with open('device_list') as f:
    device_list = f.read().splitlines()

# Iterate through device list and configure the devices
for devices in device_list:
    print ('Connecting to device ' + devices)
    ip_address_of_device = devices
    ios_device = {
        'device_type': 'cisco_ios',
        'ip': ip_address_of_device,
        'username': username,
        'password': password
    }
    # Error handling parameters
    try:
        net_connect = Netmiko(**ios_device)
    except (AuthenticationException):
        print ('Authentication failure: ' + ip_address_of_device)
        continue
    except (NetMikoTimeoutException):
        print ('Timeout to device: ' + ip_address_of_device)
        continue
    except (EOFError):
        print ("End of file while attempting device " + ip_address_of_device)
        continue
    except (SSHException):
        print ('SSH Issue. Are you sure SSH is enabled? ' + ip_address_of_device)
        continue
    except Exception as unknown_error:
        print ('Some other error: ' + str(unknown_error))
        continue

    # Configure the device and save config
    output = net_connect.send_config_from_file('config_commands')
    output += net_connect.send_command('wr mem')
    print (output)

Enter your SSH username: sdf
Enter your password: ········
Connecting to device 172.21.56.120
Authentication failure: 172.21.56.120
Connecting to device 172.21.56.121
Authentication failure: 172.21.56.121


### Task 7: Building blocks of NAPALM python script

In this exercise you will learn how a basic NAPALM python script is constructed. 

First we have to import the NAPALM connection drivers
```python
from napalm import get_network_driver
```
The connection parameters are provided to the network driver, which includes end device OS type, ip address and login credentials. 

```python
driver = get_network_driver('ios')
device = driver('172.21.56.121', 'cisco', 'cisco')
```

NAPALM will open the SSH connection using the connection paramenters

```python
device.open()
```

Once connection is established, NAPALM functions are called to perform various tasks.

```python
device.get_facts()
```

Close the SSH connection.

```python
device.close()
```

#### Primary functions of NAPALM
 
- __load_merge_candidate:__ Populate the candidate config, either from file or text.
- __load_replace_candidate:__ Similar to load_merge_candidate, but instead of a merge, the existing configuration will be entirely replaced with the content of the file, or the configuration loaded as text.
- __compare_config:__ Return the difference between the running configuration and the candidate.
- __discard_config:__ Discards the changes loaded into the candidate configuration.
- __commit_config:__ Commit the changes loaded using load_merge_candidate or load_replace_candidate.
- __rollback:__ Revert the running configuration to the previous state.

#### Addtional get functions 

- __get_facts:__ collect facts and operational data from end devices (vendor, model, uptime, etc.)
- __get_interfaces:__ speed, mac, enabled, description, etc.
- __get_interfaces_counters:__ packets, octets, errors
- __get_bgp_neighbors:__ AS, IP, received prefixes, accepted prefixes, etc.
- __get_environment:__ fan, temp, power, cpu, mem
- __get_lldp_neighbors:__ hostname, port

Note: NAPALM requires some prerequisites for properly interacting with the cisco devices. For more information, pls refer the documentation:
https://napalm.readthedocs.io/en/latest/support/ios.html


### Task 8: Collecting facts from network devices

The python script will
- login to the iosv-1 device via SSH
- run the show version command
- capture the output

In [ ]:
#!/usr/bin/env python

from napalm import get_network_driver

driver = get_network_driver('ios')
device = driver('172.21.56.121', 'cisco', 'cisco')

device.open()
print('Napalm Is Running........\n')
facts = device.get_facts()
print(facts)
device.close()

#### By default napalm get_facts will provide a strcutred data output in dictionary format. To make it more readable we can use pprint module

In [ ]:
#!/usr/bin/env python
import pprint
from napalm import get_network_driver

driver = get_network_driver('ios')
device = driver('172.21.56.121', 'cisco', 'cisco')

device.open()
print('Napalm Is Running........\n')
facts = device.get_facts()

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(facts)
device.close()

#### Use get_facts info to build a report 

In [ ]:
#!/usr/bin/env python
import pprint
from napalm import get_network_driver
from prettytable import PrettyTable

device_ip = ['172.21.56.121', '172.21.56.122']
# Using prettytable to print a sample report
report = PrettyTable(['Hostname', 'Vendor', 'Model', 'Serial No.'])

for ip in device_ip:
    driver = get_network_driver('ios')
    device = driver(ip, 'cisco', 'cisco')
    device.open()
    print('Napalm Is Running on ' + ip + '........\n')
    facts = device.get_facts()
    report.add_row([facts['hostname'], facts['vendor'], facts['model'], facts['serial_number']])
    device.close()
print (report)  

### Task 8: Configuring a network device with merge option

The python script will
- login to the iosv-1 device via SSH
- run the show version command
- capture the output

In [ ]:
!cat new_loopback.cfg

#### rLoad the new config file using 'load_merge_candidate()' and use 'compare_config()' to compare it with the running config to check what changes are required and print the diffs

In [ ]:
#!/usr/bin/env python
import pprint
from napalm import get_network_driver

driver = get_network_driver('ios')
device = driver('172.21.56.121', 'cisco', 'cisco')

device.open()
print('Napalm Is Running........\n')
device.load_merge_candidate(filename='new_loopback.cfg')
diffs = device.compare_config()

if len(diffs) > 0:
    print(diffs)
else:
    print('No changes needed')
    

#### Step 4: If in case something went worng after commiting the change and we want to rollback the change, we can use 'rollback()' to remove previously commited config changes

In [ ]:
#!/usr/bin/env python
import pprint
from napalm import get_network_driver

driver = get_network_driver('ios')
device = driver('172.21.56.121', 'cisco', 'cisco')
device.open()
print('Napalm Is Running........\n')

device.rollback()

pp = pprint.PrettyPrinter(indent=4)

facts = device.get_facts()
# print interface list

pp.pprint(facts['interface_list'])

device.close()

#### Step 4: Use the load_replace_candidate() feature to reset the lab devices to default config.

In [ ]:
#!/usr/bin/env python
import pprint
from napalm import get_network_driver

driver = get_network_driver('ios')
device = driver('172.21.56.123', 'cisco', 'cisco')
device.open()
print('Napalm Is Running........\n')

device.load_replace_candidate(filename='default.cfg')
diffs = device.compare_config()

if len(diffs) > 0:
    print(diffs)
    commit = input('Commit changes? <y or n>:')
    if commit == 'y':
        print('Commiting changes...')
        device.commit_config()
        print('Done')
    else:
        print('No changes needed')
        device.discard_config()
device.close() 